# Create a RAG Engine corpus with a Vertex AI Vector Search index

#### Import libraries and initialize client libraries

In [1]:
import os

from google import genai
import vertexai
from google.cloud import aiplatform

from vertexai import rag

PROJECT_ID = "andrewcooley-genai-tests"
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

#### Create Vertex AI Vector Search index - this is a long-running operation and could take several minutes to complete

In [2]:
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name="live_api_rag_engine_index",
    description="index for Live API with RAG engine",
    dimensions=768,
    approximate_neighbors_count=10,
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=7,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    feature_norm_type="UNIT_L2_NORM",
    index_update_method="STREAM_UPDATE",
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/511623653677/locations/us-central1/indexes/965989134722138112/operations/7737129012442431488
MatchingEngineIndex created. Resource name: projects/511623653677/locations/us-central1/indexes/965989134722138112
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/511623653677/locations/us-central1/indexes/965989134722138112')


#### Create Vertex AI Vector Search endpoint

In [3]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="index for Live API with RAG engine", public_endpoint_enabled=True
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/511623653677/locations/us-central1/indexEndpoints/7684022772619542528/operations/1344374894506278912
MatchingEngineIndexEndpoint created. Resource name: projects/511623653677/locations/us-central1/indexEndpoints/7684022772619542528
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/511623653677/locations/us-central1/indexEndpoints/7684022772619542528')


#### Deploy index to endpoint - this is a long-running operation and will take many minutes to complete

In [4]:
my_index_endpoint.deploy_index(
    index=my_index, deployed_index_id="live_api_rag_engine_index"
)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/511623653677/locations/us-central1/indexEndpoints/7684022772619542528
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/511623653677/locations/us-central1/indexEndpoints/7684022772619542528/operations/8439796107428495360
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/511623653677/locations/us-central1/indexEndpoints/7684022772619542528


resource name: projects/511623653677/locations/us-central1/indexEndpoints/7684022772619542528

#### Configure embedding model for vector-based similarity search and Vertex AI Vector Search for vector database integration in RAG Engine

In [5]:
embedding_model_config = rag.RagEmbeddingModelConfig(
    vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
        publisher_model="publishers/google/models/text-embedding-005" #this is the default model
    )
)

vector_db = rag.VertexVectorSearch(
    index=my_index.resource_name, index_endpoint=my_index_endpoint.resource_name
)

#### Create RAG Engine corpus with vector database and embedding model configured

In [6]:
rag_corpus = rag.create_corpus(
    display_name="vvs-rag-corpus-for-live-api",
    backend_config=rag.RagVectorDbConfig(
        vector_db=vector_db,
        rag_embedding_model_config=embedding_model_config,
        ),
    )

#### Upload synthetic guest policy documents for example RAG

In [7]:
rag_file = rag.upload_file(corpus_name=rag_corpus.name,
                           path="The_Grand_Horizon_Hotel_&_Resort_-_Guest_Policy_Compendium.pdf",
                           display_name="The Grand Horizon Hotel & Resort Guest Policy Compendium",
                           description="The Grand Horizon Hotel & Resort Guest Policy Compendium",
                           transformation_config=rag.TransformationConfig(
                                        rag.ChunkingConfig(chunk_size=1024, chunk_overlap=256)),
)

#### Test RAG Engine instance with an example query

In [8]:
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
        )
    ],
    text="what is the policy on late checkout?",
    rag_retrieval_config=rag.RagRetrievalConfig(
        top_k=10,
        filter=rag.utils.resources.Filter(vector_distance_threshold=0.5),
    ),
)
print(response)

contexts {
  contexts {
    source_uri: "gs://rag-rental-prod-c9eccde5-us-central1-temp-bucket/ba723bb1-fc08-40eb-aa9f-8b293dd834a0"
    text: "Fees may apply for certain requests.\r\n2. Check-In Procedures\r\n● 2.1. Check-In Time: Standard check-in time is 3:00 PM local time.\r\n● 2.2. Early Check-In: Requests for early check-in (before 3:00 PM) are subject to\r\navailability on the day of arrival. While we will make every effort to accommodate such\r\nrequests, they cannot be guaranteed. An additional fee may apply for guaranteed early\r\ncheck-in, which must be arranged in advance. Guests arriving before their room is ready\r\nare welcome to store their luggage with our Bell Desk.\r\n● 2.3. Identification: All guests are required to present valid, government-issued photo\r\nidentification (e.g., passport, driver\'s license, national ID card) upon check-in. For\r\ninternational guests, a valid passport and visa (if applicable) are required. The name on\r\nthe identification must matc